# Usage of Curio Library

## Add Curio Module to the Path

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import Necessary Libraries

In [2]:
import asyncio
from curio import Curio  # Update the import path
import cv2
from IPython.display import display, Image, clear_output
from threading import Event, Thread
import time
import ipywidgets as widgets

## Optional: Scan Devices

In [3]:
#await Curio.scan_for_devices()

## Define Constants

In [4]:
mac_address = "EC4B2DA9-6088-D0F1-BF8F-4A0271D7BBCC"
camera_ip = "192.168.137.78"
camera_port = 4747

commands = {
    'forward': b"go(1000, 1000, 1000);\n",
    'back': b"go(-1000, -1000, 1000);\n",
    'right': b"go(-1000, 1000, 1000);\n",
    'left': b"go(1000, -1000, 1000);\n",
    'stop': b"go(0, 0, 0);\n"
}

## Initialize Curio Device

In [5]:
curio = Curio(mac_address, camera_ip, camera_port)

Connected
Curio> bytearray(b'go(1000, 1000, 1000);\r\n=undefined\r\n>')
Curio> bytearray(b'go(-1000, -1000, 1000);\r\n=undefined\r\n>')
Curio> bytearray(b'go(1000, 1000, 1000);\r\n=undefined\r\n>')
Curio> bytearray(b'go(-1000, -1000, 1000);\r\n=undefined\r\n>')
Curio> bytearray(b'go(0, 0, 0);\r\n=undefined\r\n>')
Disconnected


## Widget for video display

In [6]:
video_output = widgets.Image(format='jpeg')

def display_stream():
    try:
        while not curio.stream_stop_event.is_set():
            frame = curio.get_frame()
            if frame is not None:
                _, jpeg = cv2.imencode('.jpg', frame)
                video_output.value = jpeg.tobytes()
            time.sleep(0.01)
    except Exception as e:
        print(f"Error displaying stream: {e}")

display_thread = Thread(target=display_stream)
display_thread.start()

display(video_output)

Image(value=b'', format='jpeg')

## Display control buttons in the notebook

In [7]:
def handle_command(command):
    curio.send_command(commands[command])

forward_button = widgets.Button(description="Forward")
back_button = widgets.Button(description="Back")
left_button = widgets.Button(description="Left")
right_button = widgets.Button(description="Right")
stop_button = widgets.Button(description="Stop")

forward_button.on_click(lambda x: handle_command('forward'))
back_button.on_click(lambda x: handle_command('back'))
left_button.on_click(lambda x: handle_command('left'))
right_button.on_click(lambda x: handle_command('right'))
stop_button.on_click(lambda x: handle_command('stop'))

control_box = widgets.HBox([forward_button, back_button, left_button, right_button, stop_button])
display(control_box)

## Add a stop button to stop the Curio device and clean up

In [8]:
def stop_curio():
    curio.stop()
    display_thread.join()

stop_curio_button = widgets.Button(description="Stop Curio")
stop_curio_button.on_click(lambda x: stop_curio())
display(stop_curio_button)

Button(description='Stop Curio', style=ButtonStyle())